# Technical Analysis using Ta4j

Roboquant comes out of box with support for technical analysis. It does this by having dedicate support for the popular technical analysis library Ta4j. Ta4j contains over 130 technical indicators and has a flexible engine to define your own trading rules.

In [ ]:
%use roboquant(1.6.0-SNAPSHOT)
Welcome()

In [ ]:
// We first setup the feed we'll use in the examples below
val feed = AvroFeed.sp500()

We'll now will define a strategy using indicators and rules found in the `ta4j` library and the roboquant `Ta4jStrategy` strategy class. The `Ta4jStrategy` allows to define flexible rules to generate BUY or SELL signals for assets using standard `ta4j` indicators and rules. Please note that indicators and rules from `ta4j` can be freely used and mixed, but defining strategies and back tests are done using roboquant constructs like the `Ta4jStrategy`, so not the `ta4j` back-test framework.

You can find out more on using `ta4j` on the wiki site  https://ta4j.github.io/ta4j-wiki/Trading-strategies.html or visit the `ta4j` Discord server. 

The main steps to follow are:

1. Import the ta4j packages that you want to use (in future releases of roboquant this might be done automatically) 
2. Create a `Ta4jStrategy` instance and (optional) indicate how much history we want to track
3. Define the buying and selling rules
4. Run a back test and evaluate the results (this is not specific to `ta4j` based strategies)

<div class="alert alert-block alert-info">
If you are coming from `ta4j` please note: an important difference with `ta4j` back testing and `roboquant` is that in roboquant by default the buying and selling rules apply to all assets found in a feed. So in the below example the defined strategy is applied to all the S&P 500 stocks and not just a single stock. 
</div>

In [ ]:
import org.ta4j.core.indicators.*
import org.ta4j.core.rules.*
import org.ta4j.core.indicators.helpers.*

In [ ]:
val strategy = Ta4jStrategy(maxBarCount = 30)

strategy.buy { series ->
    val closePrice = ClosePriceIndicator(series)
    val shortSma = SMAIndicator(closePrice, 5)
    val longSma = SMAIndicator(closePrice, 30)
    CrossedUpIndicatorRule(shortSma, longSma)
}

strategy.sell { series ->
    val closePrice = ClosePriceIndicator(series)
    val shortSma = SMAIndicator(closePrice, 5)
    val longSma = SMAIndicator(closePrice, 30)
    CrossedDownIndicatorRule(shortSma, longSma)
        .or(StopLossRule(closePrice, 3.0))
        .or(StopGainRule(closePrice, 2.0))
}

In [ ]:
val roboquant =  Roboquant(strategy, AccountMetric())
roboquant.run(feed)

In [ ]:
roboquant.broker.account.summary()

In [ ]:
val metricData = roboquant.logger.getMetric("account.equity")
TimeSeriesChart(metricData)

## Bollinger Bands

This is a similar Bollinger Bands strategy as the one above for Ta-Lib. 

In [ ]:
import org.ta4j.core.indicators.bollinger.BollingerBandFacade
import org.ta4j.core.indicators.helpers.ClosePriceIndicator
import org.ta4j.core.rules.CrossedUpIndicatorRule

// How big a look-back period should we use
val period = 20

val strategy = Ta4jStrategy(maxBarCount = period)

strategy.buy { series ->
    val b = BollingerBandFacade(series, period, 1.0)
    val closePrice = ClosePriceIndicator(series)
    CrossedUpIndicatorRule(closePrice, b.upper())
}

strategy.sell { series ->
    val b = BollingerBandFacade(series, period, 1.0)
    val closePrice = ClosePriceIndicator(series)
    CrossedUpIndicatorRule(closePrice, b.lower())
}


In [ ]:
val roboquant =  Roboquant(strategy, AccountMetric())
roboquant.run(feed)

In [ ]:
roboquant.broker.account.summary()

In [ ]:
val metricData = roboquant.logger.getMetric("account.equity")
TimeSeriesChart(metricData)

# Indicators

In [ ]:
// This indicator calculates the percentage bandwith of Bollinger bands
val indicator = Ta4jIndicator(maxBarCount = 100) { series ->
    val b = BollingerBandFacade(series, 20, 1.0) 
    val value = b.percentB().getValue(series.endIndex).doubleValue()
    mapOf("bandwidth" to value)
}

In [ ]:
val apple = feed.assets.getBySymbol("AAPL")
val tesla = feed.assets.getBySymbol("TSLA")
val data = feed.apply(indicator, apple, tesla)
TimeSeriesChart(data)

In [ ]:
val emaIndicator = Ta4jIndicator(maxBarCount = 30) { series ->
    val closePrice = ClosePriceIndicator(series)
    val slow = EMAIndicator(closePrice, 26).getValue(series.endIndex).doubleValue() 
    val fast = EMAIndicator(closePrice, 12).getValue(series.endIndex).doubleValue() 
    mapOf("ema.slow" to slow, "ema.fast" to fast)
}

In [ ]:
val data = feed.apply(emaIndicator, apple)
TimeSeriesChart(data)